# Long/short Trading Strategy with Support Vector Machines (SVMs)

**1. Introduction**

- The aim is to implement SVM on a trading strategy for the Luxembourg index (LUXXX) based on other indices as well as technical indicators. 
- The strategy is to take a long/short position when 0.25% change in LUXXX return is predicted. 
- The long/short position will be based on an upward or downward trending market.
- This strategy does not take into account  trading costs.

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn import svm
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.tree import DecisionTreeClassifier

- Loading the data: The data contains weekly closing prices. 

In [6]:
data_df = pd.read_csv("data/luxxx.csv")
data_df.head(3)

,Date,LUXXX,MSCI ARGENTINA,BLP ORIENTE MEDIO,MSCI AUSTRALIA,MSCI AUSTRIA,MSCI BELGIUM,MSCI BRAZIL,MSCI CANADA,MSCI CHINA,...,MSCI NORWAY,MSCI PERU,MSCI RUSSIA,MSCI SINGAPORE,MSCI SOUTH AFRICA,MSCI SPAIN,MSCI SWEDEN,MSCI SWITZERLAND,MSCI UK,MSCI USA
0,01-Jan-16,1390.716,2376.29,3525.9150,1068.79,106.70,105.38,1036.23,1663.27,59.47,...,2373.17,811.96,404.73,1507.8101,1255.75,108.40,11136.65,1148.37,1818.40,1949.70
1,08-Jan-16,1291.267,2260.85,3280.6683,1005.56,97.66,99.35,952.01,1586.18,54.63,...,2209.38,751.09,388.05,1437.6600,1177.76,101.00,10389.93,1075.30,1722.00,1831.88
2,15-Jan-16,1257.086,2217.50,3118.2981,985.38,93.54,97.32,904.64,1541.08,51.54,...,2095.60,720.23,344.45,1372.9800,1133.72,97.34,10042.32,1056.01,1692.43,1789.56


- The 'Date' column is in a string format; therefore, convert it to a datetime format

In [7]:
data_df["Date"] = pd.to_datetime(data_df["Date"])

C:\Users\Asus\AppData\Local\Temp\ipykernel_6944\939905822.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data_df["Date"] = pd.to_datetime(data_df["Date"])


- Specifying the target column - the LUXXX index. The returns of LUXXX needs to be calculated since our strategy is based on the returns over the weeks. 
- The remaining indices are converted to returns instead of their prices; to scale the features into a common unit as well as to capture movements in the prices instead.

In [13]:
target_ETF = "LUXXX"

ETF_features = data_df.loc[:, ~data_df.columns.isin(["Date", target_ETF])].columns
data_df[ETF_features] = data_df[ETF_features].pct_change()

data_df[target_ETF + "_returns"] = data_df[target_ETF].pct_change()